# Figure 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from zebrafish_ms2_paper.trace_analysis import extract_traces, compute_trace_uncertainty, enforce_1spot_per_nucleus
from zebrafish_ms2_paper.utils import pboc_rc, style_axes, colors, fontsize
import pandas as pd
import pickle
from matplotlib import rcParams

In [2]:
%matplotlib qt

In [3]:
"""specify the path to the folder of Supplemental Data Files"""
# REPLACE THIS WITH YOUR PATH
datadir = r'/home/brandon/Downloads/SupplementalDataFiles'

# define paths to particular files
path_to_spots = datadir + '/Supplemental_Data_File_2-Spots_Curated.pkl'
path_to_non_blank_timepoints = datadir + '/Supplemental_Data_File_3-Non-Blank_Time_Points.pkl'


In [4]:
fontsize = 8
linewidth = 2
markersize = 4
rcParams.update(pboc_rc)
rcParams['pdf.fonttype'] = 42

In [6]:
"""load the DataFrame with spot info and non_blank_timepoints array, which gives the correct time stamps"""
df = pd.read_pickle(path_to_spots)

with open(path_to_non_blank_timepoints,'rb') as fp:
    non_blank_timepoints = pickle.load(fp)

## Figure 2E
Plot one example trace with error bars

In [7]:
"""extract just one trace"""
good_ids = [9161]
df = df[[nuc in good_ids for nuc in df.nucleus_id]]
df = enforce_1spot_per_nucleus(df, method='gauss3d_dog')

# extract culled traces
traces = extract_traces(df, method='gauss3d_dog')

In [8]:
"""plot"""
plt.figure(figsize=(3.5, 2))
for i, trace in enumerate(traces):
    # unpack trace
    t_arr, inten_arr, nucleus = trace

    # assign correct time stamps
    t_arr = non_blank_timepoints[t_arr.astype('int')]

    # compute trace uncertainty
    sub_df = df[df.nucleus_id == nucleus]
    t_arr_bkg = sub_df.t.values
    bkg_arr = sub_df.offset.values
    t_arr_bkg = t_arr_bkg[~np.isnan(bkg_arr)]
    bkg_arr = bkg_arr[~np.isnan(bkg_arr)]
    s = compute_trace_uncertainty(t_arr_bkg, bkg_arr, inten_arr)
    l = inten_arr - s
    u = inten_arr + s

    # normalize trace to the maximum and plot
    max_inten = np.max(inten_arr)
    plt.plot(t_arr, inten_arr / max_inten, '-', linewidth=linewidth, label='MS2', color=colors['green'])
    plt.plot(t_arr, inten_arr / max_inten, 'o', label='_nolabel_', markersize=markersize, fillstyle='none', markeredgecolor='k', markeredgewidth=0.5*linewidth)
    plt.fill_between(t_arr, l / max_inten, u / max_inten, facecolor=colors['green'], alpha=0.25)

    # for clarity, only show a subset of the trace
    plt.xlim([25, 129])
    plt.ylim([-0.1, 1.3])
    plt.yticks([0, 0.5, 1])
    plt.xticks([30, 60, 90, 120])
    plt.xlabel('time (min)', fontsize=fontsize)
    plt.ylabel('\n her1-MS2 fluorescence \nintensity (a.u.)', fontsize=fontsize)
    
    # adjust plot line thicknesses
    ax = style_axes(plt.gca(), fontsize=fontsize)

plt.tight_layout()

In [21]:
plt.savefig(r'/media/brandon/Data1/Somitogenesis/Dorado/example_trace_v2.pdf')